## Задание на семинар 28 сентября по курсу анализа текстов ШАД

#### Тема: Определение частей речи с помощью скрытой марковской модели и алгоритма Витерби. 



**Дедлайн**:   9:00 утра 5 октября (для заочников 9.00 утра 8 октября)

**Среда выполнения**: Jupyter Notebook (Python 3)




В данной  работе вам предстоит:

- обучить скрытую марковскую модель на размеченных данных
- реализовать алгоритм Витерби для декодирования

Задание сдается в контест.

### Определение частей речи (POS)

Мы будем решать задачу определения частей речи (POS-теггинга) с помощью скрытой марковской модели (HMM). Формула совместной плотности наблюдаемых и скрытых переменных задается как

$$ p(X, T) = p(T) p(X|T) = p(t_1)  \prod_{i=2}^N p(t_i|t_{i-1}) \prod_{i=1}^N p(x_i|t_i)$$

В данном случае:

- наблюдаемые переменные $X$ - это слова корпуса;

- скрытые переменные $T$ - это POS-теги.

#### 1. Обучение HMM на размеченных данных

Требуется построить скрытую марковскую модель и настроить все ее параметры с помощью оценок максимального правдоподобия по размеченным данным (последовательности пар слово+тег):

- Вероятности переходов между скрытыми состояниями $p(t_i | t_{i - 1})$ посчитайте на основе частот биграмм POS-тегов.

- Вероятности эмиссий наблюдаемых состояний $p(x_i | t_i)$ посчитайте на основе частот "POS-тег - слово".

- Обратите внимание на проблему разреженности счетчиков и сделаейте все вероятности сглаженными по Лапласу (add-one smoothing).

- Распределение вероятностей начальных состояний $p(t_1)$ задайте равномерным.

Обратите внимание, что так как мы используем размеченные данные, то у нас нет необходимости в оценивании апостериорных вероятностей на скрытые переменные с помощью алгоритма forward-backword и использовании EM-алгоритма.

In [49]:
def train_hmm(tagged_sents):
    """
    Calucaltes p(tag), p(word|tag), p(tag|tag) from corpus.

    Args:
        tagged_sents: list of list of tokens. 
            Example: 
            [['I', 'go'],['She', 'goes']]

    Returns:
        p_t, p_w_t, p_t_t - tuple of 3 elements:
        p_t - dict(float), tag -> proba
        p_w_t - dict(dict(float), tag -> word -> proba
        p_t_t - dict(dict(float), previous_tag -> tag -> proba
    """
    alpha=1e-24
    counter_tag_tag = Counter()
    counter_tag_word = Counter()
    tags = set()
    p_t_t = defaultdict(dict)
    p_t = dict()
    
    sents_len = 0
    for sent in tagged_sents:
        sents_len += len(sent)
        for i, pair in enumerate(sent):
            tags.update([pair[1]])
            if i:
                counter_tag_tag[(pair[1], sent[i - 1][1])] += 1
            counter_tag_word[pair] += 1
    
    all_t_t = sents_len - len(tagged_sents)
    for (tag, prev_tag), cnt in counter_tag_tag.items():
        p_t_t[prev_tag][tag] = (cnt + alpha)/(all_t_t + alpha*len(counter_tag_tag))

    all_w_t = len(counter_tag_word) + len(tags)
    unk_prob = (1 + alpha)/(sents_len + alpha*all_w_t)
    p_w_t = defaultdict(lambda: defaultdict(lambda: unk_prob))


    for (word, tag), cnt in counter_tag_word.items():
        p_w_t[tag][word] = (cnt + alpha)/(sents_len + alpha*all_w_t)
        
    for tag in tags:
        p_t[tag] = 1 / len(tags)
    
    return p_t, p_w_t, p_t_t

In [50]:
from collections import Counter, defaultdict

import numpy as np
from nltk.corpus import brown


brown_tagged_sents = list(brown.tagged_sents(tagset="universal"))

np.random.shuffle(brown_tagged_sents)
border = int(0.9 * len(brown_tagged_sents))
train = brown_tagged_sents[:border]
test = brown_tagged_sents[border:]

Загрузите brown корпус с универсальной системой тегирования. Для этого вам понадобятся ресурсы brown и universal_tagset из nltk.download().  В этой системе содержатся следующие теги:

- ADJ - adjective (new, good, high, ...)
- ADP - adposition	(on, of, at, ...)
- ADV - adverb	(really, already, still, ...)
- CONJ	- conjunction	(and, or, but, ...)
- DET - determiner, article	(the, a, some, ...)
- NOUN	- noun	(year, home, costs, ...)
- NUM - numeral	(twenty-four, fourth, 1991, ...)
- PRT -	particle (at, on, out, ...)
- PRON - pronoun (he, their, her, ...)
- VERB - verb (is, say, told, ...)
- .	- punctuation marks	(. , ;)
- X	- other	(ersatz, esprit, dunno, ...)

Тегсеты в корпусах текстов и в различных теггерах могут быть разными. Но есть маппер: http://www.nltk.org/_modules/nltk/tag/mapping.html

Проанализируйте данные, с которыми Вы работаете. В частности, ответьте на вопросы:
- Каков общий объем датасета, формат?
- Приведены ли слова к нижнему регистру? Чем  это нам может в дальнейшем помешать?
- Как распределены слова в корпусе?  Как распределены теги в корпусе?

Сделайте случайное разбиение выборки на обучение и контроль в отношении 9:1 и обучите скрытую марковскую модель из предыдущего пункта. Если впоследствии обучение моделей будет занимать слишком много времени, работайте с подвыборкой, например, только текстами определенных категорий.

In [51]:
total_words = sum([len(sent) for sent in brown_tagged_sents])
print(total_words, 'words in dataset')
print(len(brown_tagged_sents), 'sentences in dataset')

1161192 words in dataset
57340 sentences in dataset


In [52]:
brown_tagged_sents[0]

[('The', 'DET'),
 ('late', 'ADJ'),
 ('Secretary', 'NOUN'),
 ('of', 'ADP'),
 ('State', 'NOUN'),
 ('John', 'NOUN'),
 ('Foster', 'NOUN'),
 ('Dulles', 'NOUN'),
 ('considered', 'VERB'),
 ('the', 'DET'),
 ('1954', 'NUM'),
 ('Geneva', 'NOUN'),
 ('agreement', 'NOUN'),
 ('a', 'DET'),
 ('specimen', 'NOUN'),
 ('of', 'ADP'),
 ('appeasement', 'NOUN'),
 (',', '.'),
 ('saw', 'VERB'),
 ('that', 'DET'),
 ('resolution', 'NOUN'),
 ('would', 'VERB'),
 ('be', 'VERB'),
 ('needed', 'VERB'),
 ('to', 'PRT'),
 ('keep', 'VERB'),
 ('it', 'PRON'),
 ('from', 'ADP'),
 ('becoming', 'VERB'),
 ('a', 'DET'),
 ('calamity', 'NOUN'),
 ('for', 'ADP'),
 ('the', 'DET'),
 ('West', 'NOUN'),
 ('.', '.')]

Датасет - лист предложений. Предложение - лист tuple'ов вида (слово, часть речи).

Слова не приведены к нижнему регистру. Из-за этого могут часто встречаться слова, которые были в корпусе небольшое число раз, в частности $1$. В частности поэтому лучше добавить UNK слово в p_w_t словарь, чтобы когда мы встретим незнакомое слово, оно не первратило все вероятности в $0$.

In [53]:
Counter([pair[1] for sent in brown_tagged_sents for pair in sent])

Counter({'.': 147565,
         'ADJ': 83721,
         'ADP': 144766,
         'ADV': 56239,
         'CONJ': 38151,
         'DET': 137019,
         'NOUN': 275558,
         'NUM': 14874,
         'PRON': 49334,
         'PRT': 29829,
         'VERB': 182750,
         'X': 1386})

In [54]:
Counter([pair[0] for sent in brown_tagged_sents for pair in sent])

Counter({'foremost': 12,
         'Hardware': 1,
         'raillery': 1,
         'yearning': 7,
         'rocket-bombs': 1,
         'traumatic': 1,
         'Acropolis': 6,
         'naturalness': 2,
         'surrealists': 1,
         'Bombay': 1,
         'guess': 52,
         "outskirt's": 1,
         'Marble': 3,
         'Pride': 2,
         'Murphy': 7,
         'Asteroidal': 1,
         'synonymy': 1,
         'vaster': 1,
         'abbot': 1,
         'Macklin': 8,
         'Veblen': 1,
         'mother-introject': 1,
         'AID': 4,
         "ksu'u'peli'afo": 1,
         'brothels': 2,
         'anti-monopoly': 4,
         '-': 78,
         'two-year': 5,
         'Challenge': 1,
         'carbide': 2,
         'gamut': 4,
         'pitches': 6,
         'gabbling': 1,
         'Miantonomi': 2,
         'full-sisters': 1,
         "Monday's": 3,
         'assignments': 18,
         'modulated': 1,
         'attracted': 25,
         'truncated': 3,
         'cents': 26,
  

In [56]:
p_t, p_w_t, p_t_t = train_hmm(train)

#### 2 Алгоритм Витерби для применения модели

Чтобы использовать обученную модель для определения частей речи на новых данных, необходимо реализовать алгоритм Витерби. Это алгоритм динамиеского программирования, с помощью которого мы будем находить наиболее вероятную последовательность скрытых состояний модели для фиксированной последовательности слов:

$$ \hat{T} = \arg \max_{T} p(T|X) = \arg \max_{T} p(X, T) $$

Определим функцию, определяющую максимальную вероятность последовательности, заканчивающейся на $i$-ой позиции в состоянии $k$:

$$\delta(k, i) = \max_{t_1, \dots t_{i-1}} p(x_1, \dots x_i, t_1, \dots t_i=k)$$

Тогда $\max_{k} \delta(k, N)$ - максимальная вероятность всей последовательности. А состояния, на которых эта вероятность достигается - ответ задачи.

Алгоритм Витерби заключается в последовательном пересчете функции $\delta(k, i)$ по формуле:

$$\delta(k, i) = \max_{m} \delta(m, i-1) p(t_i = k|t_{i-1} = m) p(x_i|t_i=k) $$

Аналогично пересчитывается функция, определяющая, на каком состоянии этот максимум достигается:

$$s(k, i) = \arg \max_{m} \delta(m, i-1) p(t_i = k|t_{i-1} = m) p(x_i|t_i=k) $$


На практике это означает заполнение двумерных массивов размерности: (длина последовательности) $\times$ (количество возможных состояний). Когда массивы заполнены, $\arg \max_{k} \delta(k, N)$ говорит о последнем состоянии. Начиная с него можно восстановить все состояния по массиву $s$. 

Осталось уточнить, как стартовать последовательный пересчет (чем заполнить первый столбец массива вероятностей):

$$\delta(k, 1) = p(k) p(x_1|t_1=k)$$

В реализации рекомендуется перейти к логарифмам, т.к. произведение большого числа маленьких вероятностей может приводить к вычислительным ошибкам.

In [57]:
def viterbi_algorithm(test_tokens_list, p_t, p_w_t, p_t_t):
    """
    Args:
        test_tokens_list: list of tokens. 
            Example: 
            ['I', 'go']
        p_t: dict(float), tag->proba
        p_w_t: - dict(dict(float), tag -> word -> proba
        p_t_t: - dict(dict(float), previous_tag -> tag -> proba

    Returns:
        list of hidden tags
    """
    tags = list(p_t)
    probs = np.zeros((len(tags), len(test_tokens_list)))
    max_path = np.zeros_like(probs, dtype=np.int8)

    for i, word in enumerate(test_tokens_list):
        for k, tag in enumerate(tags):
            if i:
                possible_probs = [probs[m][i - 1] +
                                  np.log(p_t_t[prev_tag][tag]) +
                                  np.log(p_w_t[tag][word])
                                  for m, prev_tag in enumerate(tags)]
                probs[k][i] = np.max(possible_probs)
                max_path[k][i] = np.argmax(possible_probs)
            else:
                probs[k][i] = np.log(p_t[tag]) + np.log(p_w_t[tag][word])

    result = [np.argmax(probs[:, -1])]

    for i in range(len(test_tokens_list) - 1, 0, -1):
        result.append(max_path[result[-1]][i])
    result = [tags[ind] for ind in reversed(result)]
    return result

Проверьте работу реализованного алгоритма на следующих модельных примерах, проинтерпретируйте результат.

- 'he can stay'
- 'a milk can'
- 'i saw a dog'
- 'an old saw'

In [59]:
examples = ['he can stay'.split(), 'a milk can'.split(),
            'i saw a dog'.split(), 'an old saw'.split()]
for example in examples:
    print(viterbi_algorithm(example, p_t, p_w_t, p_t_t))

['PRON', 'VERB', 'VERB']
['DET', 'NOUN', 'VERB']
['NOUN', 'VERB', 'DET', 'NOUN']
['DET', 'ADJ', 'VERB']


На этих примерах получилось так, что модель не очень обратила внимание на контекст и сопоставила словам более менее самые стандартные для этого слова токены. Ясно, что can гораздо чаще употребляется как глагол, нежели существительное.

Примените модель к отложенной выборке Брауновского корпуса и подсчитайте точность определения тегов (accuracy). Сделайте выводы. 

In [61]:
right_answers_count = 0
tags_number = 0

for sent in test[:1]:
    tags_number += len(sent)
    words = [pair[0] for pair in sent]
    tags = [pair[1] for pair in sent]
    predicted_tags = viterbi_algorithm(words, p_t, p_w_t, p_t_t)
    for right, predicted in zip(tags, predicted_tags):
        if right == predicted:
            right_answers_count += 1

print('accuracy', right_answers_count / tags_number)

accuracy 1.0
